In [13]:
import os
import numpy as np
import xarray as xr

os.chdir("/Users/etmu9498/research/code/scripts")
import tc_metadata
import make_plots
import helper_fns

In [63]:
# goal: save all tc eyewall limit times in a dictionary of dictionaries!
# outer dict = year, inner dict = eye pass
# ex: eyewalls for 9/27/21 = eyewall_times[ '2021'][ '0927']

eyewall_times = {}
eyewall_times[ '2021'] = {}

# do this for all tcs!
tclist = ['fred', 'grace', 'henri', 'ida', 'sam']
for tci, tcname in enumerate( tclist):
    print( 'TC ' + tcname)
    
    # load 2021 metadata for this tc
    metadata = tc_metadata.all_data( tc = tcname)
    # print( metadata.keys())
    # print( 'tc path: ' + metadata['um_crl_path'])

    # make a new list to hold all eyewall lims for this date
    prev_date = ''
    date_list = []
    
    # do this for all the eye pass datasets for this tc
    for namei in range( len( metadata['eyewall_dists_no_eyewalls'])):

        # load data and print helpful notices for the user
        tdrname, crlname =  tc_metadata.choose_new_data( tcname, counter = namei)
        print( 'pass ' + str( namei))
        print( 'name: ' + crlname)
        print( 'date: ' + metadata[ 'dates'][ namei])
        wall_dists = metadata['eyewall_dists_no_eyewalls'][ namei]
        print( 'eyewall lims: ' + str( wall_dists))
        os.chdir( metadata['um_crl_path'])
        data = xr.open_dataset( crlname)

        # find and print the times closest to the eyewall limits!
        left_lim_i = np.argmin( np.abs( np.array( data.in_situ_distance) - wall_dists[0] ))
        right_lim_i = np.argmin( np.abs( np.array( data.in_situ_distance) - wall_dists[1] ))

        left_time = np.round( data.time[ left_lim_i].values.item(), 4)
        right_time = np.round( data.time[ right_lim_i].values.item(), 4)

        print( left_time)
        print( str( right_time) )
        
        
        # put the date into a nicer format
        if len( metadata['dates'][namei]) == 8:
            date_reformat = metadata['dates'][namei][0:2] + metadata['dates'][namei][3:5] + metadata['dates'][namei][6:8]
        else:
            date_reformat = metadata['dates'][namei][0:2] + metadata['dates'][namei][3:5]        
        
        # add eyewall times to overall dictionary!
        # first date case
        if len( date_list) == 0:
            
            # DON'T add the old dictionary, just make a new one
            date_list.append( (left_time, right_time) )
            
            # iterate the previous date holder
            prev_date = metadata['dates'][namei]
            
        # new date case, existing dataset
        elif prev_date != metadata['dates'][namei]:
            
            # add old dictionary to the overall dict!
            eyewall_times['2021'][ prev_date] = date_list
            
            # make a new dict and repeat
            date_list = [ (left_time, right_time) ]
            
            # iterate the previous date holder
            prev_date = metadata['dates'][namei]

        # repeat date case
        else:
            date_list.append( (left_time, right_time) )

            # iterate the previous date holder
            prev_date = metadata['dates'][namei]
            
            
        # last case: add the data we have now!
        if namei == len( metadata['eyewall_dists_no_eyewalls']) - 1:
            eyewall_times['2021'][ date_reformat] = date_list
        print( '\n')

            


TC fred
pass 0
name: crl-fred-08-12-am-eye-1.nc
date: 08-12-am
eyewall lims: (-10, 40)
10.4787
10.5821


pass 1
name: crl-fred-08-12-am-eye-2.nc
date: 08-12-am
eyewall lims: (-70, -16)
10.9949
11.106


pass 2
name: crl-fred-08-12-am-eye-3.nc
date: 08-12-am
eyewall lims: (-55, 50)
12.3615
12.5788


pass 3
name: crl-fred-08-12-pm-eye-1.nc
date: 08-12-pm
eyewall lims: (-45, 47.5)
22.1608
22.3563


pass 4
name: crl-fred-08-12-pm-eye-2.nc
date: 08-12-pm
eyewall lims: (-52.5, -11)
23.3269
23.4141


pass 5
name: crl-fred-08-13-eye-1.nc
date: 08-13
eyewall lims: (-57.5, 40)
9.6456
9.8534


pass 6
name: crl-fred-08-13-eye-2.nc
date: 08-13
eyewall lims: (-55, 50)
10.9701
11.194


TC grace
pass 0
name: crl-grace-08-16-eye-1.nc
date: 08-16
eyewall lims: (-53, -36)
10.9583
10.9938


pass 1
name: crl-grace-08-16-eye-2.nc
date: 08-16
eyewall lims: (25, 55)
12.3583
12.4216


pass 2
name: crl-grace-08-17-eye-1.nc
date: 08-17
eyewall lims: (-65, 70)
10.0851
10.369


pass 3
name: crl-grace-08-17-eye-2.nc

In [73]:
print( eyewall_times)

#for keyi, keyval in enumerate( eyewall_times['2021'].keys() ):   
#    # print( keyval)
#    print( str( eyewall_times['2021'][keyval]))

{'2021': {'08-12-am': [(10.4787, 10.5821), (10.9949, 11.106), (12.3615, 12.5788)], '08-12-pm': [(22.1608, 22.3563), (23.3269, 23.4141)], '0813': [(9.6456, 9.8534), (10.9701, 11.194)], '08-16': [(10.9583, 10.9938), (12.3583, 12.4216)], '08-17': [(10.0851, 10.369), (11.4268, 11.5113), (12.868, 12.9924)], '08-18': [(21.8553, 21.9103), (23.1903, 23.2798), (24.3673, 24.4395)], '0819': [(22.0095, 22.0995), (22.9457, 23.0707), (24.0498, 24.2193)], '08-20': [(21.9232, 22.0104), (24.5149, 24.581)], '0821': [(22.5069, 22.5875), (23.7514, 23.823), (25.9095, 26.0095)], '08-27': [(21.519, 21.669), (23.5469, 23.6558), (24.7547, 24.8097), (26.7216, 26.8372)], '0829': [(18.9966, 19.0349)], '09-26': [(22.6209, 22.6826), (23.8448, 23.8937), (24.8804, 24.9521)], '09-27': [(23.3316, 23.3971), (24.5296, 24.5724)], '0929': [(21.5788, 21.6793), (22.7988, 22.8877)]}}


In [72]:
eyewall_time_limits = {}

# part 1: get times from the 2021 eye pass data
# look under "2023-02-27 get 2021 eyewall limits.ipynb" to see how I wrote some automatic code
# to find these limits!
eyewall_time_limits[ '2021'] = {}

# date and time eye pass metadata
dates = ['08-12-am', '08-12-pm', '0813',
        '08-16', '08-17', '08-18',
        '0819', '08-20', '0821',
        '08-27', '0829', '09-26',
        '09-27', '0929']
passes = [
    [ (10.4787, 10.5821), (10.9949, 11.106), (12.3615, 12.5788)],
    [(22.1608, 22.3563), (23.3269, 23.4141)],
    [(9.6456, 9.8534), (10.9701, 11.194)],
    [(10.9583, 10.9938), (12.3583, 12.4216)],
    [(10.0851, 10.369), (11.4268, 11.5113), (12.868, 12.9924)],
    [(21.8553, 21.9103), (23.1903, 23.2798), (24.3673, 24.4395)],
    [(22.0095, 22.0995), (22.9457, 23.0707), (24.0498, 24.2193)],
    [(21.9232, 22.0104), (24.5149, 24.581)],
    [(22.5069, 22.5875), (23.7514, 23.823), (25.9095, 26.0095)],
    [(21.519, 21.669), (23.5469, 23.6558), (24.7547, 24.8097), (26.7216, 26.8372)],
    [(18.9966, 19.0349)],
    [(22.6209, 22.6826), (23.8448, 23.8937), (24.8804, 24.9521)],
    [(23.3316, 23.3971), (24.5296, 24.5724)],
    [(21.5788, 21.6793), (22.7988, 22.8877)]
    ]
# add metadata to the dictionary!
for datei, dateval in enumerate( dates):
    eyewall_time_limits['2021'][ dateval] = passes[ datei]

# part 2: add 2022 eye pass data
eyewall_time_limits[ '2022'] = {}

dates = []
passes = [

]

print( eyewall_time_limits)

{'2021': {'08-12-am': [(10.4787, 10.5821), (10.9949, 11.106), (12.3615, 12.5788)], '08-12-pm': [(22.1608, 22.3563), (23.3269, 23.4141)], '0813': [(9.6456, 9.8534), (10.9701, 11.194)], '08-16': [(10.9583, 10.9938), (12.3583, 12.4216)], '08-17': [(10.0851, 10.369), (11.4268, 11.5113), (12.868, 12.9924)], '08-18': [(21.8553, 21.9103), (23.1903, 23.2798), (24.3673, 24.4395)], '0819': [(22.0095, 22.0995), (22.9457, 23.0707), (24.0498, 24.2193)], '08-20': [(21.9232, 22.0104), (24.5149, 24.581)], '0821': [(22.5069, 22.5875), (23.7514, 23.823), (25.9095, 26.0095)], '08-27': [(21.519, 21.669), (23.5469, 23.6558), (24.7547, 24.8097), (26.7216, 26.8372)], '0829': [(18.9966, 19.0349)], '09-26': [(22.6209, 22.6826), (23.8448, 23.8937), (24.8804, 24.9521)], '09-27': [(23.3316, 23.3971), (24.5296, 24.5724)], '0929': [(21.5788, 21.6793), (22.7988, 22.8877)]}, '2022': {}}
